<a href="https://colab.research.google.com/github/lilfetz22/Running_data/blob/Joining-Notes-columns/Running_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Running Log Analysis**

###Goals


By 9/17 - complete pre-Garmin Data Wrangling
1. check for any remaining information in Notes columns - done (9/17)
2. Fix Date column and fill it in and make it agree with weekday
3. remove excess columns

By 10/1 - Complete post-Garmin analysis
1. Data Wrangling

    a. filtering out activities from one another 

    b. comparing overlap from pre-garmin to post-Garmin records and fill in missing data

# Pre-Garmin Data Wrangling

In [1]:
import pandas as pd
import re
import numpy as np
import math

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
def findall_n_join_fx(df, note_col, new_col, search_string):
    df.loc[:,new_col] = df.loc[:,note_col].str.findall(search_string)
    df.loc[:,new_col] = df.loc[:,new_col].str.join('')
    df.loc[:,note_col] = df.loc[:,note_col].str.replace(search_string, '')
    return df

In [4]:
log_csv = pd.read_csv("/content/drive/My Drive/pre-garmin.csv",encoding='windows-1254')
log_csv.head(15)

,Running log 6/21/10,3/17/14,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Saturday 5/29/10,NaN,4 miles 27:19 it was a little short. 3.8 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monday 6/21/10,4 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday 6/22/10,4 miles,Swimming,10 laps of freestyle,5 laps of backstroke,5,laps of breaststroke,NaN,1 lap all out of freestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wednesday 6/23/10,4.5 miles,35 minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thursday 6/25/10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Friday 6/26/10,5,Swimming,same as Tuesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Saturday 6/27/10,6,41:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Monday 6/28/10,4,28:04:00,5 total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tuesday 6/29/10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
log_csv.tail()

,Running log 6/21/10,3/17/14,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
1979,TOTAL - 48.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,Monday - 12/21/15 - off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,Tuesday - I've failed. Not going to be able to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,Thursday - 12/25/15 - 3 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,Thursday - 12/31/15 - 30 minutes biking - 9.5 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creation of Raw string before manipulation of columns

In [6]:
log_csv.columns

Index(['Running log 6/21/10 ', ' 3/17/14', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')

In [7]:
raw_string = log_csv.copy()
raw_string['raw_note'] = raw_string['Running log 6/21/10 '].str.cat(raw_string[[' 3/17/14','Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7',
                                                     'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11','Unnamed: 12',
                                                     'Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17']].astype(str), sep="_")
raw_string2 = raw_string[['raw_note']]
raw_string2.loc[:, 'raw_note'] = raw_string2.loc[:, 'raw_note'].str.replace('_?nan_?', '')
raw_string2.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,raw_note
0,Saturday 5/29/10 4 miles 27:19 it was a little...
1,Monday 6/21/10_ 4 miles
2,Tuesday 6/22/10_ 4 miles_Swimming_ 10 laps of ...
3,Wednesday 6/23/10_ 4.5 miles_35 minutes
4,Thursday 6/25/10_ 5


In [8]:
log_csv['Notes'] = log_csv['Unnamed: 2'].str.cat(log_csv[['Unnamed: 3', 'Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7',
                                                     'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11','Unnamed: 12',
                                                     'Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17']].astype(str), sep="_")
log_csv2 = log_csv[['Running log 6/21/10 ',' 3/17/14','Notes']]
log_csv2.head(10)

,Running log 6/21/10,3/17/14,Notes
0,Saturday 5/29/10,NaN,4 miles 27:19 it was a little short. 3.8 mile...
1,Monday 6/21/10,4 miles,NaN
2,Tuesday 6/22/10,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...
3,Wednesday 6/23/10,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...
4,Thursday 6/25/10,5,NaN
5,Friday 6/26/10,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...
6,Saturday 6/27/10,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...
7,NaN,NaN,NaN
8,Monday 6/28/10,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...
9,Tuesday 6/29/10,5,NaN


In [ ]:
# columns = ['Weekday', 'Date', 'Mileage', 'Minutes', 'Notes']
log_csv2.columns = ['Beginning', 'Mileage', 'Notes']

In [ ]:
m = '- 7 miles - warmup with maggie - 82.190 - 6:19:16 - 13.0 avg'
t = '- 8 miles - thanksgiving 18.229 mile'
e = '35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan'
re.sub('_nan', '', e)

'35 minutes'

In [ ]:
findall_n_join_fx(log_csv2, 'Beginning', 'Weekday', '\w+day')
log_csv2 = findall_n_join_fx(log_csv2, 'Beginning', 'Date', '\d+/\d+/\d\d')
log_csv2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Beginning  1966 non-null   object
 1   Mileage    1217 non-null   object
 2   Notes      751 non-null    object
 3   Weekday    1966 non-null   object
 4   Date       1966 non-null   object
dtypes: object(5)
memory usage: 77.6+ KB


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [ ]:
log_csv2.sample(20)

,Beginning,Mileage,Notes,Weekday,Date
1097,TOTAL,44,NaN,,
1386,TOTAL - 52 miles,NaN,NaN,,
392,,9/19/11,2 on 2 off around shelly _ then tempo for 10:...,Monday,
1276,TOTAL - 22 miles - 4 ab workouts,NaN,NaN,,
594,,2 mile warmups,2 mile _ 10:22 _ 5:00 first mile _ 4 total_na...,Saturday,
1391,- 7 miles - 82.629 - 6:35:20 - 12.5 avg,NaN,NaN,Friday,
1517,- 4 miles,NaN,NaN,Friday,
1454,- - 10 miles,NaN,NaN,Monday,9/8/14
1852,BIKE TOTAL -,NaN,NaN,,
889,,mile warmup,out ray road _ 11 miles _ 12 total_nan_nan_na...,Saturday,


In [ ]:
log_csv3 = log_csv2[log_csv2['Beginning'].str.startswith('TOTAL') == False]

In [ ]:
log_csv3.loc[:,'Mileage_init'] = log_csv3.loc[:,'Beginning'].str.findall('\d+\.*\d*')
log_csv3.loc[:, 'length'] = log_csv3['Mileage_init'].str.len()
log_csv3['counter'] = range(len(log_csv3))
log_csv3['Workout_mins'] = np.nan
log_csv3['Swimming_meters'] = np.nan
log_csv3['Bike_Mileage'] = np.nan
log_csv3 = log_csv3.rename(columns = {'Beginning': 'Notes2'})
check_data = log_csv3[log_csv3.length >1]
check_data2 = check_data[['Notes2','Mileage','Mileage_init','counter']]
len(check_data2)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

157

In [ ]:
check_data2 = check_data2.reset_index()

In [ ]:
# log_csv3.to_csv("/content/drive/My Drive/mileage_manip.csv")

In [9]:
new_log = pd.read_csv("/content/drive/My Drive/mileage_manip.csv",index_col=0)
new_log.head(10)

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,[],0,0,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,[],0,1,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,[],0,2,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,[],0,3,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,[],0,4,NaN,NaN,NaN
5,,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...,Friday,6/26/10,[],0,5,NaN,NaN,NaN
6,,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Saturday,6/27/10,[],0,6,NaN,NaN,NaN
8,,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...,Monday,6/28/10,[],0,7,NaN,NaN,NaN
9,,5,NaN,Tuesday,6/29/10,[],0,8,NaN,NaN,NaN
10,,5.5,Swimming_normal_nan_nan_nan_nan_nan_nan_nan_na...,Wednesday,6/30/10,[],0,9,NaN,NaN,NaN


In [10]:
new_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1723 entries, 0 to 1983
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1665 non-null   object 
 1   Mileage          1216 non-null   object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             139 non-null    object 
 5   Mileage_init     1712 non-null   object 
 6   length           1723 non-null   int64  
 7   counter          1723 non-null   int64  
 8   Workout_mins     10 non-null     float64
 9   Swimming_meters  18 non-null     float64
 10  Bike_Mileage     112 non-null    float64
dtypes: float64(3), int64(2), object(6)
memory usage: 161.5+ KB


In [11]:
to_be_nan = list(new_log.counter[new_log.length > 1])
for row_numb in to_be_nan:
    new_log.iloc[row_numb,5] = np.nan

In [12]:
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\[\'', '')
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\'\]', '')
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\[\]', '')

In [13]:
new_log.Mileage_init = pd.to_numeric(new_log.Mileage_init, errors='coerce')
new_log.head()

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,NaN,0,1,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,NaN,0,4,NaN,NaN,NaN


In [14]:
new_log.insert(8, 'Workout_type',None)

In [15]:
new_log.head()

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN


In [16]:
# dropping two lines that just had "WORKOUTS TOTAL"
new_log = new_log.drop([1727,1747])
new_log['counter'] = range(len(new_log))

In [17]:
def workout_finder_fx(df, col, str_search, minutes, workout_type, *exclude):
    w_o = list(df.loc[:, col].str.contains(str_search))
    for idx, n in enumerate(w_o):
        if n == None:
            continue
        elif n == True:
            enough_to_exclude = 0
            for e in exclude:
                o2 = re.search(e, df[col].iloc[idx])
                if o2:
                    enough_to_exclude += 1
            if enough_to_exclude < 1:
                df['Workout_type'].iloc[idx] = workout_type
                df['Workout_mins'].iloc[idx] = minutes
    return df



In [18]:
ab_workout_incl_log = workout_finder_fx(new_log, 'Notes2', '[Aa]b [Ww]orkout', 20, 'abs')
gym_workout_incl_log = workout_finder_fx(ab_workout_incl_log, 'Notes2', '[Ww]orkout',45, 'gym', '[Ss]wim', '[Aa]b')
print(gym_workout_incl_log.iloc[1084])
gym_workout_incl_log.iloc[1267]

Notes2              - 4 miles - ab workout - ran with Keith
Mileage                                                 NaN
Notes                                                   NaN
Weekday                                             Tuesday
Date                                                    NaN
Mileage_init                                              4
length                                                    1
counter                                                1084
Workout_type                                            abs
Workout_mins                                             20
Swimming_meters                                         NaN
Bike_Mileage                                            NaN
Name: 1264, dtype: object


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Notes2              - 14 miles - workout between classes
Mileage                                              NaN
Notes                                                NaN
Weekday                                          Tuesday
Date                                                 NaN
Mileage_init                                          14
length                                                 1
counter                                             1267
Workout_type                                         gym
Workout_mins                                          45
Swimming_meters                                      NaN
Bike_Mileage                                         NaN
Name: 1472, dtype: object

Mileage Column

In [19]:
gym_workout_incl_log.iloc[314]

Notes2                                                              
Mileage                                                     8/29/11 
Notes               warmup _ 20 mins inside _ 2.5ish _ 2 with gir...
Weekday                                                       Monday
Date                                                             NaN
Mileage_init                                                     NaN
length                                                             0
counter                                                          314
Workout_type                                                    None
Workout_mins                                                     NaN
Swimming_meters                                                  NaN
Bike_Mileage                                                     NaN
Name: 371, dtype: object

In [20]:
# might need to try out the "try" in a function to see if it can be converted to 
# an integer if not, then put the note into a separate column and delete it from
# the mileage column
def remove_mileage_notes_fx(df, col):
    notes3_col = [''] * len(df[col])
    list_col = list(df.loc[:, col])
    for idx, a in enumerate(list_col):
        try:
            float_a = float(a)
            df[col].iloc[idx] = float_a
        except:
            notes3_col[idx] = a
            df[col].iloc[idx] = np.nan
    df['Notes3'] = notes3_col
    return df
sep_notes_from_mileage_col = remove_mileage_notes_fx(gym_workout_incl_log, 'Mileage')
sep_notes_from_mileage_col.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN,
1,,NaN,NaN,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN,4 miles
2,,NaN,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,NaN,NaN,4 miles
3,,NaN,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN,4.5 miles
4,,5,NaN,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN,
5,,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...,Friday,6/26/10,NaN,0,5,None,NaN,NaN,NaN,
6,,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Saturday,6/27/10,NaN,0,6,None,NaN,NaN,NaN,
8,,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...,Monday,6/28/10,NaN,0,7,None,NaN,NaN,NaN,
9,,5,NaN,Tuesday,6/29/10,NaN,0,8,None,NaN,NaN,NaN,
10,,5.5,Swimming_normal_nan_nan_nan_nan_nan_nan_nan_na...,Wednesday,6/30/10,NaN,0,9,None,NaN,NaN,NaN,


took all of the text that was in the mileage column and moved it to a separate column, and left any number that was purely a number and converted the column to be integers. 

In [21]:
# now I need to go through the notes3 column and pull out any that have just mileage, 
#then I will need to go through and search those that have dates and other mess 
#in there, then I will need to apply the same functions to the Notes column

In [22]:
findall_n_join_fx(sep_notes_from_mileage_col, 'Notes3', 'Date2', '\d+/\d+/\d\d')

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3,Date2
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN,,
1,,NaN,NaN,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN,4 miles,
2,,NaN,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,NaN,NaN,4 miles,
3,,NaN,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN,4.5 miles,
4,,5,NaN,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,- 10 miles,NaN,NaN,Sunday,NaN,10.0,1,1716,None,NaN,NaN,NaN,,
1980,- - off,NaN,NaN,Monday,12/21/15,NaN,0,1717,None,NaN,NaN,NaN,,
1981,- I've failed. Not going to be able to run fo...,NaN,NaN,Tuesday,NaN,NaN,0,1718,None,NaN,NaN,NaN,,
1982,- - 3 miles,NaN,NaN,Thursday,12/25/15,3.0,1,1719,None,NaN,NaN,NaN,,


In [23]:
sep_notes_from_mileage_col.loc[34]

Notes2                                   
Mileage                               NaN
Notes                                 NaN
Weekday                         Wednesday
Date                              7/28/10
Mileage_init                          NaN
length                                  0
counter                                29
Workout_type                         None
Workout_mins                          NaN
Swimming_meters                       NaN
Bike_Mileage                          NaN
Notes3             2 miles / 5 miles hike
Date2                                    
Name: 34, dtype: object

In [24]:
def date_finder_fx(df, new_date):
    date_col = list(df[new_date])
    for idx, date in enumerate(date_col):
        try:
            if math.isnan(date):
                continue
        except:
            if date == '':
                continue
            try:
                if math.isnan(df['Date'].iloc[idx]):
                    df['Date'].iloc[idx] = date
            except:
                continue
                

In [25]:
date_finder_fx(sep_notes_from_mileage_col, 'Date2')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
sep_notes_from_mileage_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             295 non-null    object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     68 non-null     object 
 9   Workout_mins     70 non-null     float64
 10  Swimming_meters  18 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
 13  Date2            1721 non-null   object 
dtypes: float64(4), int64(2), object(8)
memory usage: 281.7+ KB


In [27]:
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('_nan', '')

In [28]:
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.lower()
sep_notes_from_mileage_col.loc[:, 'Notes2'] = sep_notes_from_mileage_col.loc[:, 'Notes2'].str.lower()
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.lower()

In [29]:
sep_notes_from_mileage_col['Mileage_init2'] = sep_notes_from_mileage_col.Notes3.str.findall('\d+\.?\d* miles?')
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.replace('\d+\.?\d* miles?', '')
sep_notes_from_mileage_col['Mileage_init3'] = sep_notes_from_mileage_col.Notes.str.findall('\d+\.?\d* miles?')
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('\d+\.?\d* miles?', '')
sep_notes_from_mileage_col['Mileage_init4'] = sep_notes_from_mileage_col.Notes3.str.findall('\d+\.?\d* total')
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.replace('\d+\.?\d* total', '')
sep_notes_from_mileage_col['Mileage_init5'] = sep_notes_from_mileage_col.Notes.str.findall('\d+\.?\d* total')
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('\d+\.?\d* total', '')

In [30]:
gym_workout_incl_log2 = workout_finder_fx(sep_notes_from_mileage_col, 'Notes', 'workout', 45, 'gym', 'swim', 'ab','pool')
gym_workout_incl_log2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             295 non-null    object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     73 non-null     object 
 9   Workout_mins     75 non-null     float64
 10  Swimming_meters  18 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
 13  Date2            1721 non-null   object 
 14  Mileage_init2    1721 non-null   object 
 15  Mileage_init3    741 non-null    object 
 16  Mileage_init4    1721 non-null   object 
 17  Mileage_init5 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
def swim_workout_fx(df, col, str_search): 
    for s in str_search:
        w_o2 = list(df.loc[:, col].str.contains(s))
        for idx, w in enumerate(w_o2):
            if w == None:
                continue
            elif w == True:
                df['Swimming_meters'].iloc[idx] = 1600

In [32]:
swim_workout_fx(gym_workout_incl_log2, 'Notes', ['swim','pool'])
swim_workout_fx(gym_workout_incl_log2, 'Notes2', ['swim','pool'])
swim_workout_fx(gym_workout_incl_log2, 'Notes3', ['swim','pool'])
gym_workout_incl_log2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             295 non-null    object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     73 non-null     object 
 9   Workout_mins     75 non-null     float64
 10  Swimming_meters  39 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
 13  Date2            1721 non-null   object 
 14  Mileage_init2    1721 non-null   object 
 15  Mileage_init3    741 non-null    object 
 16  Mileage_init4    1721 non-null   object 
 17  Mileage_init5 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
def mileage_finder_fx(df, init, total_or_miles):
    multiple_miles = []
    too_big = []
    already_in_col = []
    for idx, row in enumerate(df.loc[:, init]):
        try:
            if math.isnan(row):
                continue
        except:
            if len(row) > 1:
                multiple_miles.append(idx)
            elif len(row) == 1:
                mile_str = row[0].replace(total_or_miles, '')
                try:
                    mile_float = float(mile_str)
                    if math.isnan(df['Mileage'].iloc[idx]):
                        if mile_float > 27.2:
                            too_big.append(idx)
                        else:
                            df['Mileage'].iloc[idx] = mile_float
                    else:
                        already_in_col.append(idx)
                except:
                    continue
    return multiple_miles, too_big, already_in_col



In [34]:
mm3, tb3, aic3 = mileage_finder_fx(gym_workout_incl_log2,'Mileage_init4','total')
print(len(mm3),len(tb3),len(aic3))
mm4, tb4, aic4 = mileage_finder_fx(gym_workout_incl_log2, 'Mileage_init5', 'total')
print(len(mm4),len(tb4),len(aic4))
mm, tb, aic = mileage_finder_fx(gym_workout_incl_log2,'Mileage_init2','miles')
print(len(mm),len(tb),len(aic))
mm2, tb2, aic2 = mileage_finder_fx(gym_workout_incl_log2, 'Mileage_init3', 'miles')
print(len(mm2),len(tb2),len(aic2))

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0 0 0
1 0 3
5 3 36
67 0 89


For each of the following I will be manually putting in the data to fix the issue since they are outliers in being able to conform nicely to a computer function. 

In [35]:
comb = mm4 + aic4 + mm + tb + aic + mm2 + aic2
comb_array = np.array(comb)
comb_uni = np.unique(comb_array)

In [36]:
time_added = gym_workout_incl_log2.copy()
time_added.insert(2, 'Time', None)

In [37]:
raw_string_added = pd.merge(time_added, raw_string2, how='left',left_index=True, right_index=True)

In [38]:
def manual_input_desc(df, list_name):
    for ind in list_name:
        print(df[['Mileage','Bike_Mileage', 'Time','Weekday', 'Date','Workout_type', 'Workout_mins', 'Swimming_meters']].iloc[ind])
        print('RAW: ', df['raw_note'].iloc[ind])

In [39]:
manual_input_desc(raw_string_added, mm4)

Mileage                   2
Bike_Mileage            NaN
Time                   None
Weekday            Thursday
Date                    NaN
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 339, dtype: object
RAW:  Thursday _ warmup _ 2 miles _ 5:30, 5:54 _ 11:24 total time_ 43rd place _ 25ish min cooldown _ 6.5 total


In [40]:
manual_input_desc(raw_string_added, aic4)

Mileage                  4
Bike_Mileage           NaN
Time                  None
Weekday             Monday
Date               6/28/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 8, dtype: object
RAW:  Monday 6/28/10_4_28:04:00_5 total
Mileage                5.5
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date               7/13/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 23, dtype: object
RAW:  Tuesday 7/13/10_5.5_ 35:32_6.5 total
Mileage                  12
Bike_Mileage            NaN
Time                   None
Weekday            Saturday
Date                    NaN
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 729, dtype: object
RAW:  Saturday _12_13_ 10 on actual run to the beach but went down to watch the seahawk invitational _12_13 total


In [41]:
manual_input_desc(raw_string_added, mm)

Mileage                  NaN
Bike_Mileage             NaN
Time                    None
Weekday            Wednesday
Date                 7/28/10
Workout_type            None
Workout_mins             NaN
Swimming_meters          NaN
Name: 34, dtype: object
RAW:  Wednesday 7/28/10_2 miles / 5 miles hike
Mileage                NaN
Bike_Mileage           NaN
Time                  None
Weekday             Monday
Date               8/16/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 53, dtype: object
RAW:  Monday 8/16/10_3 miles in the morning 5 miles in the evening 
Mileage                  6
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date                   NaN
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 505, dtype: object
RAW:  Tuesday _ 3 miles and 24 miles bike and 3 around greystone _ 6 total
Mileage                   15
Bike_Mileage             NaN
Time           

In [42]:
manual_input_desc(raw_string_added, tb)

Mileage             NaN
Bike_Mileage        NaN
Time               None
Weekday             NaN
Date                NaN
Workout_type       None
Workout_mins        NaN
Swimming_meters     NaN
Name: 1013, dtype: object
RAW:  BIKE TOTAL _ 30 miles 
Mileage             NaN
Bike_Mileage        NaN
Time               None
Weekday             NaN
Date                NaN
Workout_type       None
Workout_mins        NaN
Swimming_meters     NaN
Name: 1021, dtype: object
RAW:  BIKE TOTAL _ 30 miles 
Mileage             NaN
Bike_Mileage        NaN
Time               None
Weekday             NaN
Date                NaN
Workout_type       None
Workout_mins        NaN
Swimming_meters     NaN
Name: 1029, dtype: object
RAW:  BIKE TOTAL _ 30 miles 


In [43]:
manual_input_desc(raw_string_added, aic)

Mileage                    5
Bike_Mileage             NaN
Time                    None
Weekday            Wednesday
Date                  7/7/10
Workout_type            None
Workout_mins             NaN
Swimming_meters          NaN
Name: 17, dtype: object
RAW:  Wednesday 7/7/10_4 miles_28:38:00_5 total
Mileage                  4
Bike_Mileage           NaN
Time                  None
Weekday             Friday
Date               7/30/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 36, dtype: object
RAW:  Friday 7/30/10_ 3 miles_19:42_4 total
Mileage                  7
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date                   NaN
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 40, dtype: object
RAW:  Tuesday_6 miles 40:47_7 total
Mileage                  7
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date                   NaN
W

In [44]:
manual_input_desc(raw_string_added, mm2)

Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Saturday
Date                5/29/10
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 0, dtype: object
RAW:  Saturday 5/29/10 4 miles 27:19 it was a little short. 3.8 miles
Mileage                 2
Bike_Mileage          NaN
Time                 None
Weekday            Friday
Date                  NaN
Workout_type         None
Workout_mins          NaN
Swimming_meters       NaN
Name: 43, dtype: object
RAW:  Friday_2 miles_fun run "Popsicle stick run"_2 miles cool down_6.5 miles total
Mileage                NaN
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date                   NaN
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 223, dtype: object
RAW:  Tuesday _ stadium mile _ 7 miles _ 9 miles total
Mileage                  NaN
Bike_Mileage             NaN
Time         

In [45]:
manual_input_desc(raw_string_added, aic2)

Mileage                   5
Bike_Mileage            NaN
Time                   None
Weekday            Thursday
Date                 7/1/10
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 11, dtype: object
RAW:  Thursday 7/1/10_5_32:04:00_6:20 pace_6 miles total
Mileage                  4
Bike_Mileage           NaN
Time                  None
Weekday            Tuesday
Date                7/6/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 16, dtype: object
RAW:  Tuesday 7/6/10_4 miles_29:14:00_5 miles total
Mileage                  6
Bike_Mileage           NaN
Time                  None
Weekday             Monday
Date               7/12/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 22, dtype: object
RAW:  Monday 7/12/10_6_42ish_7 miles
Mileage                  2
Bike_Mileage           NaN
Time                  None
Weekday             Monday
Date              

In [46]:
mil = list(raw_string_added.Mileage_init)
multi = []
for idx, m in enumerate(mil):
    if math.isnan(m):
        continue
    elif math.isnan(raw_string_added['Mileage'].iloc[idx]):
        raw_string_added['Mileage'].iloc[idx] = m
    else:
        multi.append(idx)
len(multi)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0

Search for more data inside of the Notes columns

In [47]:
raw_string_added['Notes'] = raw_string_added['Notes'].replace(np.nan, '')
raw_string_added['Notes2'] = raw_string_added['Notes2'].replace(np.nan, '')

In [48]:
raw_string_added['Notes_comb'] = raw_string_added['Notes'].str.cat(raw_string_added[['Notes2','Notes3']].astype(str), sep="_")
raw_string_added.head()

,Notes2,Mileage,Time,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3,Date2,Mileage_init2,Mileage_init3,Mileage_init4,Mileage_init5,raw_note,Notes_comb
0,,NaN,None,27:19 it was a little short.,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN,,,[],"[4 miles, 3.8 miles]",[],[],Saturday 5/29/10 4 miles 27:19 it was a little...,27:19 it was a little short. _ _
1,,4,None,,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN,,,[4 miles],NaN,[],NaN,Monday 6/21/10_ 4 miles,_ _
2,,4,None,swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,1600.0,NaN,,,[4 miles],[],[],[],Tuesday 6/22/10_ 4 miles_Swimming_ 10 laps of ...,swimming_ 10 laps of freestyle_5 laps of backs...
3,,4.5,None,35 minutes,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN,,,[4.5 miles],[],[],[],Wednesday 6/23/10_ 4.5 miles_35 minutes,35 minutes_ _
4,,5,None,,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN,,,[],NaN,[],NaN,Thursday 6/25/10_ 5,_ _


In [49]:
raw_string_added['numbers'] = raw_string_added.Notes_comb.str.findall('\d+\.*\d*')

In [50]:
greater_2 = []
two = []
one = []
for idx, num in enumerate(raw_string_added.numbers):
    if (len(num) > 2) and (idx not in comb_uni) and (math.isnan(raw_string_added['Mileage'].iloc[idx])):
        greater_2.append(idx)
    elif (len(num) == 2) and (idx not in comb_uni) and (math.isnan(raw_string_added['Mileage'].iloc[idx])):
        two.append(idx)
    elif (len(num) == 1) and (idx not in comb_uni) and (math.isnan(raw_string_added['Mileage'].iloc[idx])):
        one.append(idx)
print(len(greater_2), len(two), len(one))



48 22 48


In [51]:
manual_input_desc(raw_string_added, greater_2)

Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Saturday
Date                7/10/10
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 20, dtype: object
RAW:  Saturday 7/10/10_8miles _55:53:00
Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Saturday
Date                    NaN
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 100, dtype: object
RAW:  Saturday_ haganstone meet ran 17:16 first mile: 5:07 or 5:27, 2nd mile 11:05/ 5:38 or 5:58, 3rd mile 16:36/ 5:31, 40 sec 200 = 17:16
Mileage               NaN
Bike_Mileage          NaN
Time                 None
Weekday            Monday
Date                  NaN
Workout_type         None
Workout_mins          NaN
Swimming_meters       NaN
Name: 102, dtype: object
RAW:  Monday_ 3 min on 2 min off X 6 then 20 min cooldown
Mileage                NaN
Bike_M

In [52]:
manual_input_desc(raw_string_added, two)

Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Thursday
Date                7/22/10
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 28, dtype: object
RAW:  Thursday 7/22/10_over 5_36 min
Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Saturday
Date                    NaN
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 181, dtype: object
RAW:  Saturday _ 8.5 64 mins
Mileage                  NaN
Bike_Mileage             NaN
Time                    None
Weekday            Wednesday
Date                     NaN
Workout_type            None
Workout_mins             NaN
Swimming_meters          NaN
Name: 206, dtype: object
RAW:  Wednesday _ 6 at 630 pace 
Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Thursday
Date                    NaN
Workout

In [53]:
manual_input_desc(raw_string_added, one)

Mileage                 NaN
Bike_Mileage            NaN
Time                   None
Weekday            Thursday
Date                7/15/10
Workout_type           None
Workout_mins            NaN
Swimming_meters         NaN
Name: 25, dtype: object
RAW:  Thursday 7/15/10_30 minutes 
Mileage                NaN
Bike_Mileage           NaN
Time                  None
Weekday             Sunday
Date               7/25/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 31, dtype: object
RAW:  Sunday 7/25/10_25 minutes at state course Pool workout
Mileage                NaN
Bike_Mileage           NaN
Time                  None
Weekday             Monday
Date               11/1/10
Workout_type          None
Workout_mins           NaN
Swimming_meters        NaN
Name: 137, dtype: object
RAW:  Monday_ 11/1/10 _6.5_ got to go to the top of the building across from Harris teeter!!! It was sooo awesome 
Mileage                  NaN
Bike_Mileage             NaN
Ti

need to look at every row that is null for mileage and see whether it should be null or not

In [54]:
l_comb_uni = list(comb_uni)

In [55]:
comb_2 = l_comb_uni + greater_2 + two + one

In [56]:
missing_mileage = []
for idx, w in enumerate(raw_string_added['Mileage']):
    if (math.isnan(w)) and (idx not in comb_2):
        missing_mileage.append(idx)

In [71]:
manual_changes = {'Mileage': [[339,6.5],[8,5],[729,13],[23,6.5],[34,2],[53,8],[1247,21],[0,3.8],[43,6.5],[223,9],[373,5],[383,9],[514,9],[534,7],[542,13],[547,9],[551,7],[591,9],[756,9],[832,6],[901,10],
                          [903,8],[908,10.5],[910,8.19],[924,11.5],[1074,8],[1099,9],[1102,13.1],[1107,8],[1108,11.3],[1114,11],[1122,7],[1247,21],[1255,26.2],[11,6],[16,5],[22,7],[32,9.25],[39,4],
                          [55,6],[64,4],[88,5],[164,8],[20, 8],[50,4],[100,4.2],[102,6.5],[125,4.2],[181,8.5],[186, 10.7],[218,4],[222,8],[241,4],[252,5.5],[264,7],[304,10],[316,10.5],[337,8],[358,11],
                          [359,10],[371,8],[394,4.2],[412,7.5],[419,5.5],[426,4],[428,6],[447,3],[493,9.5],[541,10.5],[543,5.5],[544,5],[560,9.5],[572,4.5],[582,9],[601,9.5],[670,6],[699,5.5],[714,11],
                          [28,5.5],[181,8.5],[206,6],[303,2.5],[382,4],[483,10.5],[484,10.5],[486,13],[491,16],[498,16],[511,11.5],[537,7.5],[584,11],[587,9],[632,9.5],[642,4],[665,7.5],[25,4],[31,3.5],
                          [137,6.5],[139,3],[202,2],[219,7.5],[285,6],[287,2.5],[289,5],[302,2.5],[326,3],[327,4.5],[332,1.5],[374,4.5],[379,4],[439,1],[443,3],[444,3],[445,3],[446,3],[450,3],[465,13],
                          [472,15],[488,3],[490,10.5],[500,8.5],[540,7],[548,4.2],[605,2],[624,7.5],[636,4.5],[637,3],[643,5],[688,4.5],[738,8.5],[747,13],[750,19],[857,3],[862,2],[933,7],[978,4],[1093,4],
                          [1100,8],[1127,1],[1130,2],[1243,13],[134,1],[233,3],[281,3],[469,10],[601,10],[966,1]], 
                  'Time': [[8, '28:04'],[23, '35:32'],[558,'40:00'],[357,'66:01'],[351,'70:00'],[166,'34:00'],[107,'75:00'],[90,'39:00'],[63,'53:40'],[61,'50:50'],[48,'42:59'],[40,'40:47'],[36,'19:42'],
                           [17,'28:38'],[0,'27:19'],[11,'32:04'],[37,'47:13'], [20, '55:53'], [181,'64:00'],[186, '68:00'],[222,'56:37'],[316,'71:23'],[412,'16:31'],[582,'60:00'],[714,'61:10'],
                           [181,'64:00'],[537,'51:00'],[584,'80:00'],[632,'66:00'],[642,'30:00'],[665,'60:00'],[31,'25:00'],[219,'54:00']],
                  'Date': [[95, '9/20/10'], [110, '10/4/10']],
                  'Bike_Mileage': [[505,24],[1013,30],[1021,30],[1029,30],[54,14.9],[37,14.9],[328,20.7],[1362,102.21],[303,14.8],[405,30],[311,23],[452,37],[615,41]],
                  'Swimming_meters': [[31,1600],[677,1600]],
                  'Workout_type': [[214,'gym'],[1440,'gym'],[1457,'gym'],[1601,'gym']],
                  'Workout_mins': [[214,30],[1440,60],[1457,60],[1601,60]]}

In [72]:
manual_inputs_df = raw_string_added.copy()

In [73]:
def input_manual_fx(df, loc_dict):
    for key, value in loc_dict.items():
        for v in value:
            df.loc[v[0], key] = v[1] 

In [74]:
input_manual_fx(manual_inputs_df, manual_changes)

In [76]:
manual_inputs_df['Mileage'] = manual_inputs_df['Mileage'].replace(np.nan, 0)

#### Time to mess with the Date. 

6

In [77]:
date_list = list(manual_inputs_df.Date)
for idx, d in enumerate(date_list):
    try:
        if math.isnan(d):
            continue
    except:
        print(d, idx)

5/29/10 0
6/21/10 1
6/22/10 2
6/23/10 3
6/25/10 4
6/26/10 5
6/27/10 6
6/28/10 7
6/29/10 8
6/30/10 9
7/1/10 10
7/2/10 11
7/3/10 12
7/5/10 13
7/6/10 14
7/7/10 15
7/8/10 16
7/9/10 17
7/10/10 18
7/12/10 19
7/13/10 20
7/14/10 21
7/15/10 22
7/16/10 23
7/22/10 24
7/25/10 26
7/26/10 27
7/27/10 28
7/28/10 29
7/29/10 30
7/30/10 31
7/31/10 32
8/16/10 45
8/23/10 51
9/1/10 57
9/6/10 63
9/20/10 75
10/4/10 88
11/1/10 111
11/15/10 117
11/22/10 122
11/29/10 128
12/6/10 133
12/13/10 138
12/20/10 143
1/3/10 150
1/10/11 156
1/17/11 158
1/24/11 163
1/31/11 168
2/7/11 173
2/14/11 179
2/21/11 184
2/28/11 189
3/7/11 195
3/14/11 200
3/28/11 205
4/4/11 210
4/19/11 215
4/25/11 218
5/2/11 223
5/9/11 229
5/16/11 235
5/23/11 241
5/30/11 247
6/6/11 253
6/13/11 258
6/20/11 264
6/27/11 270
7/4/11 274
7/11/11 280
7/24/11 283
7/31/11 289
8/7/11 294
8/15/11 301
8/22/11 308
8/29/11 314
9/5/11 320
9/12/11 326
9/19/11 332
9/26/11 338
10/3/11 345
10/10/11 352
10/17/11 358
10/24/11 363
10/31/11 369
11/14/11 376
11/21/11 382
1

# Garmin Data Wrangling

# Future Garmin Data Web Scraping

In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

In [ ]:
URL = "https://connect.garmin.com/modern/activities"

requests.get(URL)

<Response [200]>

In [ ]:
requests.get(URL, {}).text

'<!DOCTYPE html><html><head><meta charSet="utf-8"/><meta http-equiv="x-ua-compatible" content="ie=edge"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><style data-href="/styles.3bfbd3dd9436b814ee1a.css">body,html{height:auto;font:400 14px/22px Open Sans,Helvetica Neue,Helvetica,Arial,sans-serif;color:#222;background-color:#efefef;margin:0}a,a:active,a:hover,a:visited{text-decoration:none;color:inherit}#truste-consent-track{display:block}#truste-consent-content{display:flex;align-items:center;justify-content:center;flex-wrap:wrap;width:100%;box-sizing:border-box}#truste-consent-text{margin:0 1rem 0 0}#truste-consent-buttons{display:flex;flex-direction:row;margin-top:1px!important;margin-left:10px!important}#truste-consent-button,#truste-show-consent{font-family:Open Sans,HelveticaNeue,HelveticaNeueu,Arial,sans-serif;font-size:14px;color:#fff;background-color:#4a4a4a;cursor:pointer;padding:5px 20px;margin:12px 0;border-radius:4px;border:none}#trus

In [ ]:
web_page = bs4.BeautifulSoup(requests.get(URL, {}).text, "lxml")

In [ ]:
web_page

<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><style data-href="/styles.3bfbd3dd9436b814ee1a.css">body,html{height:auto;font:400 14px/22px Open Sans,Helvetica Neue,Helvetica,Arial,sans-serif;color:#222;background-color:#efefef;margin:0}a,a:active,a:hover,a:visited{text-decoration:none;color:inherit}#truste-consent-track{display:block}#truste-consent-content{display:flex;align-items:center;justify-content:center;flex-wrap:wrap;width:100%;box-sizing:border-box}#truste-consent-text{margin:0 1rem 0 0}#truste-consent-buttons{display:flex;flex-direction:row;margin-top:1px!important;margin-left:10px!important}#truste-consent-button,#truste-show-consent{font-family:Open Sans,HelveticaNeue,HelveticaNeueu,Arial,sans-serif;font-size:14px;color:#fff;background-color:#4a4a4a;cursor:pointer;padding:5px 20px;margin:12px 0;border-radius:4px;border:none}#trus

In [ ]:
web_page.head.title

<title data-react-helmet="true">Garmin Connect | </title>

In [ ]:
web_page.body

<body><noscript id="gatsby-noscript">This app works best with JavaScript enabled.</noscript><div id="___gatsby"><div role="group" style="outline:none" tabindex="-1"><div><header class="c017"><h1 class="c0116 c0118"><a class="c018" href="/" title="Connect"></a></h1><div class="c0114"></div></header><main class="c014 c016"><div class="c013"><div class="c011"><div class="c012" id="gauth-widget"><noscript>To sign in to your Garmin account, please enable JavaScript in your web browser.</noscript></div></div><div id="loginGauth"></div></div></main><footer class="c0123"><div></div><div class="c0132"><span class="c0129 c0133"><a href="https://www.instagram.com/garminfitness/" title="Instagram"></a></span><span class="c0130 c0134"><a href="https://www.facebook.com/garminfit" title="Facebook"></a></span><span class="c0131 c0135"><a href="https://twitter.com/garminfitness" title="Twitter"></a></span></div><div id="consent"></div></footer></div></div></div><script id="gatsby-script-loader">/*<![CD

In [ ]:
sub_webpage = web_page.find_all(name="li") #, attrs={'class': 'list-item animated row-fluid'}

In [ ]:
sub_webpage

[]

I would like to make searchable notes that I could go through with links to exercises for injuries and such. It would be really useful to be able to have the types of injuries that I have gone through before and be able to just immediately have the stretches and such that worked. Then I can the next time find out how that went and then adjust based upon the previous injury timeline and see what works best for recovery for me. 